In [39]:
%pip install -qU langchain langchain-openai python-dotenv rich langchain_postgres langchain_community beautifulsoup4 langchain_experimental "psycopg[binary]"

Note: you may need to restart the kernel to use updated packages.


In [51]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [57]:
import os
database_url = os.getenv("DATABASE_URL")
if not database_url:
    print("⚠️  DATABASE_URL not found in environment variables.")
else:
    print(f"✅ DATABASE_URL found: {database_url[:20]}...")  # Show first 20 chars for privacy


✅ DATABASE_URL found: postgresql+psycopg:/...

In [85]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
chat_model = ChatOpenAI(model="o4-mini")

In [54]:
import bs4
from rich import print
from langchain_community.document_loaders import WebBaseLoader
from langchain_experimental.text_splitter import SemanticChunker

# Load and chunk contents of the documentation
loader = WebBaseLoader(
    web_paths=("https://python.langchain.com/docs/introduction/",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(name="article")),
)
document = loader.load()
print(document[0].page_content[:500])

IntroductionOn this pageIntroduction
LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Development: Build your applications using LangChain's open-source components and third-party integrations.
Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support.
Productionization: Use LangSmith to inspect, monitor and evaluate your applications, so that you

In [55]:
# Semantic Chunking
text_splitter = SemanticChunker(embeddings=embeddings, breakpoint_threshold_type="interquartile")
chunks = text_splitter.split_documents(document)
print(chunks[0])

Document(
    metadata={'source': 'https://python.langchain.com/docs/introduction/'},
    page_content="IntroductionOn this pageIntroduction\nLangChain is a framework for developing applications 
powered by large language models (LLMs). LangChain simplifies every stage of the LLM application 
lifecycle:\n\nDevelopment: Build your applications using LangChain's open-source components and third-party 
integrations. Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support. 
Productionization: Use LangSmith to inspect, monitor and evaluate your applications, so that you can continuously 
optimize and deploy with confidence. Deployment: Turn your LangGraph applications into production-ready APIs and 
Assistants with LangGraph Platform. LangChain implements a standard interface for large language models and 
related\ntechnologies, such as embedding models and vector stores, and integrates with\nhundreds of providers."
)

In [56]:
from langchain_postgres import PGVector
import os

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="my_docs",
    connection=os.getenv("DATABASE_URL"),
    create_extension=True,
)

vector_store.add_documents(documents=chunks)
print(f"Added {len(chunks)} documents to the vector store")

Added 4 documents to the vector store

In [89]:
from langchain_core.prompts import ChatPromptTemplate

hyde_prompt = ChatPromptTemplate.from_template(
    """
Given the question '{question}', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth. The document size has be within {chunk_size} words.
"""
)
hyde_prompt.format(question="what is langchain?", chunk_size=500)

"Human: \nGiven the question 'what is langchain?', generate a hypothetical document that directly answers this question. The document should be detailed and in-depth. The document size has be within 500 words.\n"

In [91]:
from rich import print
llm_chain = hyde_prompt | chat_model
question = "what is Langgraph?"
generated_docs = llm_chain.invoke({"question": question, "chunk_size":100}).content
print(generated_docs)

Langgraph is a hypothetical AI-driven semantic graph framework designed to represent, interlink, and query natural 
language constructs across multilingual datasets. It models linguistic units—words, phrases, entities, syntactic 
relations—as nodes and edges, enriched with embeddings capturing semantic, syntactic, and contextual nuances. By 
integrating ontologies, vector representations, and provenance metadata, Langgraph enables advanced tasks like 
cross-lingual information retrieval, contextual search, entity resolution, and reasoning over textual corpora. Its 
API supports graph-based queries, incremental learning, and real-time updates, fostering scalable NLP applications 
ranging from intelligent assistants to domain-specific knowledge discovery and automated summarization.

In [92]:
retrieved_docs = vector_store.similarity_search_with_score(question+generated_docs, k=2)
print(retrieved_docs)

[
    (
        Document(
            id='8b571d4d-c929-49c2-9417-7406a1ec8fb4',
            metadata={'source': 'https://python.langchain.com/docs/introduction/'},
            page_content="IntroductionOn this pageIntroduction\nLangChain is a framework for developing 
applications powered by large language models (LLMs). LangChain simplifies every stage of the LLM application 
lifecycle:\n\nDevelopment: Build your applications using LangChain's open-source components and third-party 
integrations. Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support. 
Productionization: Use LangSmith to inspect, monitor and evaluate your applications, so that you can continuously 
optimize and deploy with confidence. Deployment: Turn your LangGraph applications into production-ready APIs and 
Assistants with LangGraph Platform. LangChain implements a standard interface for large language models and 
related\ntechnologies, such as embedding models and vector stores, and integrates with\nhundreds of providers."
        ),
        0.3512722849845886
    ),
    (
        Document(
            id='5d3b157f-729c-406f-90c9-8864906ec11f',
            metadata={'source': 'https://python.langchain.com/docs/introduction/'},
            page_content="Architecture\u200b\nThe LangChain framework consists of multiple open-source libraries. 
Read more in the\nArchitecture page. langchain-core: Base abstractions for chat models and other components. 
Integration packages (e.g. langchain-openai, langchain-anthropic, etc.): Important integrations have been split 
into lightweight packages that are co-maintained by the LangChain team and the integration developers. langchain: 
Chains, agents, and retrieval strategies that make up an application's cognitive architecture. langchain-community:
Third-party integrations that are community maintained. langgraph: Orchestration framework for combining LangChain 
components into production-ready applications with persistence, streaming, and other key features. See LangGraph 
documentation. Guides\u200b\nTutorials\u200b\nIf you're looking to build something specific or are more of a 
hands-on learner, check out our tutorials section. This is the best place to get started. These are the best ones 
to get started with:\n\nBuild a Simple LLM Application\nBuild a Chatbot\nBuild an Agent\nIntroduction to 
LangGraph\n\nExplore the full list of LangChain tutorials here, and check out other LangGraph tutorials here. To 
learn more about LangGraph, check out our first LangChain Academy course, Introduction to LangGraph, available 
here. How-to guides\u200b\nHere you’ll find short answers to “How do I….?” types of questions."
        ),
        0.3907158374786377
    )
]